In [ ]:
from IPython.display import Audio

In [5]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Load the NSynth dataset
dataset, info = tfds.load('nsynth', split='valid', with_info=True)




Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...: 100%|██████████| 4/4 [00:00<00:00, 85.34 url/s]
Extraction completed...: 0 file [00:00, ? file/s]
Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]usage: ipykernel_launcher.py [-h] [--dataflow_endpoint DATAFLOW_ENDPOINT]
                             [--project PROJECT] [--job_name JOB_NAME]
                             [--staging_location STAGING_LOCATION]
                             [--temp_location TEMP_LOCATION] [--region REGION]
                             [--service_account_email SERVICE_ACCOUNT_EMAIL]
                             [--no_auth]
                             [--template_location TEMPLATE_LOCATION]
                             [--label LABELS] [--update]
                             [--transform_name_mapping TRANSFORM_NAME_MAPPING]
                             [--enable_streaming_engine]
                             [--dataflow_kms_key DATAFLOW_KMS_KEY]
                             [--create_from

**************************** WARNING *********************************
yet no `beam_runner` nor `beam_options` was explicitly provided.

Some Beam datasets take weeks to generate, so are usually not suited
for single machine generation. Please have a look at the instructions
to setup distributed generation:

https://www.tensorflow.org/datasets/beam_datasets#generating_a_beam_dataset
**********************************************************************


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [ ]:
# Iterate over the dataset
import librosa
import librosa.display
import matplotlib.pyplot as plt

filtered_dataset = dataset.filter(lambda x: x['instrument']['family'] == 10)

for example in filtered_dataset.take(1):  # Only take a single example
    audio = example['audio'].numpy()
    print(example)
    # rate = example['sample_rate']
    label = example['instrument']['family']


sample_rate = 16000
# stft = librosa.stft(audio)
# # Convert the STFT to a spectrogram
# spectrogram = librosa.amplitude_to_db(abs(stft))

# print(spectrogram.shape)
# # Plot the spectrogram
# plt.figure(figsize=(10, 4))
# librosa.display.specshow(spectrogram, sr=sample_rate, x_axis='time', y_axis='hz')
# plt.colorbar(format='%+2.0f dB')
# plt.title('Spectrogram')
# plt.show()

# Compute MFCCs
mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)

# Plot the MFCCs
print(mfccs.shape)
plt.figure(figsize=(10, 4))
librosa.display.specshow(mfccs, sr=sample_rate, x_axis='time')
plt.colorbar()
plt.title('MFCC')
plt.tight_layout()
plt.show()

print(label)
Audio(audio, rate=sample_rate)

In [ ]:
import librosa
import numpy as np
from sklearn.decomposition import NMF
import tensorflow as tf

# Let's assume 'audio' is your loaded audio signal and 'sample_rate' is the sampling rate
filtered_dataset = dataset.filter(lambda x: x['instrument']['family'] == 1)
x_train = dict()
y_train = dict()

for i in range(10):
  filtered_dataset = dataset.filter(lambda x: x['instrument']['family'] == i)
  x_train = {**x_train, **{f"{i}":[]}}
  y_train = {**y_train, **{f"{i}":[]}}
  for sample in filtered_dataset:
    audio = sample['audio'].numpy()
    noise = np.random.normal(0,0.01, audio.shape)
    noisy_audio = noise + audio
    stft = librosa.stft(noisy_audio)
    magnitude, phase = librosa.magphase(stft)
    x_train[i].append(np.flatten(magnitude))
    stft = librosa.stft(audio)
    magnitude, phase = librosa.magphase(stft)
    y_train[i].append(np.flatten(magnitude))

sample_rate = 16000
input_shape = x_train['0'][0].shape
input_shape_flatten = input_shape[0] * input_shape[1]
n_components = 2

model = tf.keras.Sequential([
    tf.keras.layers.Dense(input_shape=input_shape_flatten, n_components*input_shape[1]),
    tf.keras.layers.Activation(tf.keras.activations.softplus),
    tf.keras.layers.Dense(input_shape_flatten),
    tf.keras.layers.Activation(tf.keras.activations.softplus)
])

def loss_function(y_true, y_pred):
    input_len = y_true.shape[0]
    loss = 0
    for i in range(input_len):
      loss += y_true[i]*(np.log(y_true[i]) - np.log(y_pred[i])) - y_true[i] + y_pred[i]

    return loss


# Apply NMF to the magnitude of the STFT (since phase information is not non-negative)

model = NMF(n_components=n_components, init='random', random_state=0)
W = model.fit_transform(magnitude)
H = model.components_


# # Reconstruct the magnitude from NMF components
# idx = 0
# reconstructed_magnitude = np.dot(W[:,idx], H[idx])

# # Combine with original phase
# reconstructed_stft = reconstructed_magnitude * phase

# Inverse STFT to get the audio signal
# reconstructed_audio = librosa.istft(reconstructed_stft)
reconstructed_sources = []
for i in range(n_components):
    # Reconstruct the magnitude for each source
    source_magnitude = np.outer(W[:, i], H[i])

    # Combine with original phase information
    source_stft = source_magnitude * phase

    # Inverse STFT to get the time-domain signal of each source
    source_audio = librosa.istft(source_stft)
    reconstructed_sources.append(source_audio)

# reconstructed_audio is the approximation of the original audio after NMF decomposition and reconstruction


In [ ]:
print(H.shape)

In [ ]:
Audio(reconstructed_sources[0], rate=sample_rate)

In [ ]:
Audio(audio, rate=sample_rate)

In [ ]:
Audio(noisy_audio, rate=sample_rate)

In [ ]:
import tensorflow as tf
